In [ ]:
# installed accelerate

In [48]:
import pandas as pd
import datasets

In [2]:
data1 = pd.read_csv("cte/tomasz_test/data2.csv")

In [112]:
ds = datasets.Dataset.from_pandas(data1)

In [108]:
#ds = datasets.load_dataset('csv', data_files='cte/tomasz_test/data2.csv')

In [113]:
ds

Dataset({
    features: ['Unnamed: 0', 'Repository', 'Message', 'Diff'],
    num_rows: 540
})

In [56]:
ds = ds.map(tokenize, num_proc=4, batched=True)

In [3]:
data1

,Unnamed: 0,Repository,Message,Diff
0,0,swaroopch/byte-of-python,Remove Romanian translation (abandoned)\n\nThe...,diff --git a/translations.md b/translations.md...
1,1,swaroopch/byte-of-python,Add link to Real Python on editor,diff --git a/first_steps.md b/first_steps.md\n...
2,2,swaroopch/byte-of-python,Fix typo,diff --git a/README.md b/README.md\nindex 85a9...
3,3,swaroopch/byte-of-python,Remove whitespace to fix header rendering\n\nT...,diff --git a/problem_solving.md b/problem_solv...
4,4,swaroopch/byte-of-python,Fix sentence grammar\n\nThanks to John Thomas.,diff --git a/basics.md b/basics.md\nindex 3729...
...,...,...,...,...
535,535,purcellconsult/Cracking-Python-Bootcamp,Create a list of written assignments\n\nWritte...,diff --git a/written assignments/a list of wri...
536,536,purcellconsult/Cracking-Python-Bootcamp,Create text_based_calculator\n\nThe first codi...,diff --git a/coding projects/text_based_calcul...
537,537,purcellconsult/Cracking-Python-Bootcamp,Create 1_numbers_in_python\n\nPower point file...,diff --git a/power_points/1_numbers_in_python ...
538,538,purcellconsult/Cracking-Python-Bootcamp,Add files via upload\n\nFirst lesson of the co...,diff --git a/1_numbers_in_python.py b/1_number...


In [102]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration, AutoModelForSeq2SeqLM, AutoModelForCausalLM, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer, TFTrainingArguments

In [5]:
# Model types and sizes
MODEL_SIZES = ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]
MODEL_SIZE = MODEL_SIZES[0]
MODEL_TYPES = [TFT5ForConditionalGeneration, AutoModelForSeq2SeqLM, AutoModelForCausalLM ]
MODEL_TYPE = MODEL_TYPES[0]

# Data
FEATURES = "Diff"
TARGET = "Message"
DATA = data1

# Determine task for teh model (summarize, translate)
PREFIX = "summarize: "

# Determine maximum input for features and the target
MAX_INPUT_LENGTH = 540
MAX_TARGET_LENGTH = 540

# Fine-Tuning Parameters
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
ADAM_BETA1 = 0.9
WEIGHT_DECAY = 0.01

# Testing
GIT_DIFF = "random diff that is going to be used to generate comment when committed"

In [6]:
# create the model
model = MODEL_TYPE.from_pretrained(MODEL_SIZE)
model

2023-08-30 10:08:26.948685: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-30 10:08:26.951830: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-30 10:08:26.951862: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-30 10:08:26.952573: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow 

In [7]:
# create the tokenizer
tokenizer = T5Tokenizer.from_pretrained(MODEL_SIZE)
tokenizer

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5Tokenizer(name_or_path='t5-small', vocab_size=32100, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_4

In [8]:
DATA.dtypes

Unnamed: 0     int64
Repository    object
Message       object
Diff          object
dtype: object

In [9]:
DATA[FEATURES] = DATA[FEATURES].apply(lambda x : str(x))

In [10]:
DATA[TARGET] = DATA[TARGET].apply(lambda x : str(x))

In [11]:
DATA[TARGET] = DATA[TARGET].apply(lambda x : x.replace('\n',' ').replace('.','').replace(',','').replace('(','').replace(')','').replace('!','').replace('+','').replace(':','').replace('/',''))

In [12]:
DATA.dtypes

Unnamed: 0     int64
Repository    object
Message       object
Diff          object
dtype: object

In [76]:
def preprocess_function(df):
    # Setup the tokenizer for features
    inputs = [PREFIX + difference for difference in df[FEATURES][0:10]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=list(df[TARGET][0:10]), max_length=MAX_TARGET_LENGTH, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [77]:
data3 = preprocess_function(DATA)

In [46]:
type(data3)

transformers.tokenization_utils_base.BatchEncoding

In [44]:
data3.set_format(type='pandas')

AttributeError: 

In [30]:
pd.DataFrame(data3)

,0
0,input_ids
1,attention_mask
2,labels


In [78]:
new_df =pd.DataFrame()
new_df["input_ids"] = data3["input_ids"]
new_df["attention_mask"] = data3["attention_mask"]
new_df["labels"] = data3["labels"]

In [79]:
new_df

,input_ids,attention_mask,labels
0,"[21603, 10, 20624, 1636, 12651, 3, 9, 87, 7031...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10002, 3871, 29, 7314, 13876, 37, 3871, 29, 1..."
1,"[21603, 10, 20624, 1636, 12651, 3, 9, 87, 1467...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2334, 1309, 12, 2977, 20737, 30, 6005, 1]"
2,"[21603, 10, 20624, 1636, 12651, 3, 9, 87, 1322...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[14269, 23042, 1]"
3,"[21603, 10, 20624, 1636, 12651, 3, 9, 87, 1930...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10002, 872, 6633, 12, 2210, 13956, 18968, 133..."
4,"[21603, 10, 20624, 1636, 12651, 3, 9, 87, 4883...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[14269, 7142, 19519, 1333, 12, 1079, 3576, 1]"
5,"[21603, 10, 20624, 1636, 12651, 3, 9, 87, 7, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2210, 23042, 1]"
6,"[21603, 10, 20624, 1636, 12651, 3, 9, 87, 5, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[617, 126, 747, 44, 414, 13, 689, 1]"
7,"[21603, 10, 20624, 1636, 12651, 3, 9, 87, 5, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2270, 3, 12651, 3191, 127, 15, 1]"
8,"[21603, 10, 20624, 1636, 12651, 3, 9, 87, 7932...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2334, 1030, 81, 128, 703, 1999, 2099, 257, 11..."
9,"[21603, 10, 20624, 1636, 12651, 3, 9, 87, 5, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[14269, 6854, 2196, 57, 3, 476, 4815, 10376, 7..."


In [92]:
#Function splitting data
def split_data(data):
    train_data = data.iloc[0: int(len(data)*0.7)]
    validation_data = data.iloc[int(len(data)*0.7): int(len(data)*0.9)]
    test_data = data.iloc[int(len(data)*0.9): len(data)-1]
    return train_data, validation_data, test_data

In [93]:
#Split data
train_data, validation_data, test_data = split_data(new_df)

In [105]:
# Create new training model based on T5 model and given new parameters
def create_training_model(model, tokenizer, train_data, validation_data):

    # create arguments for training the model
    args = TFTrainingArguments(
        f"{MODEL_SIZE}-finetuned-xsum",
        evaluation_strategy = "epoch",
        learning_rate=LEARNING_RATE,
        per_device_train_batch_size= BATCH_SIZE,
        per_device_eval_batch_size= BATCH_SIZE,
        weight_decay=WEIGHT_DECAY,
        adam_beta1 = ADAM_BETA1,
        save_total_limit=3,
        num_train_epochs=1,
        #predict_with_generate=True,
        fp16=True,
        push_to_hub=False)

    # pad inputs and labels to the maximum length in the batch
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    # model creation
    trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=validation_data,
    data_collator=data_collator,
    tokenizer=tokenizer)
    # compute_metrics=compute_metrics not sure whether it is needed so not included

    # return new training model
    return trainer

In [107]:
#Commit to Excellence model
CTE_model = create_training_model(model, tokenizer, train_data, validation_data)

NameError: name 'accelerate_version' is not defined

In [ ]:
# train CTE_model
CTE_model.train()

In [ ]:
#Testing

def generate_comment(tokenizer, git_diff):
    preprocessed_text= PREFIX + git_diff

    encoding_test = tokenizer.encode(preprocessed_text,return_tensors="tf")

    summary_ids = CTE_model.generate(encoding_test, min_length=60, max_length=80)

    commit_message = tokenizer.decode(summary_ids[0])

    return commit_message

In [ ]:
generate_comment(tokenizer, GIT_DIFF)
a